# A (Possible) Code for Matching Timepix Clusters to Tagger Hits

First, we'll import the tools we'll need:

In [1]:
import Acqu as aq
import AcquDetector as aqdet
import Timepix
import ROOT
import numpy as np
import pandas as pd
import root_numpy
from rootpy.plotting import histogram, Hist2D, Hist, Canvas

Welcome to JupyROOT 6.16/00


Import the data to be analyzed and the tagger calibration file (?):

In [2]:
inFile = '/scratch/2019-05_Timepix/Timepix_33.dat'
aq.openFile(inFile)

#aqdet.LoadDetectors(["taggerWithOffsets.json"])

Mk2 Data


Set up some histograms:

In [160]:
hist_posA = Hist2D(256,0,256,256,0,256,name='posA')

And now, for the fun stuff:

In [ ]:
triggerTime = []
scalerRepeat = 0
eventA  = 0    # event number for timepix A
prevTriggerTime = [0]

#global counters for cluster info
counter = 0
hits    = 0
group   = -1


sumx = 0.0
sumy = 0.0
meanx = 0.0
meany = 0.0
gn = 1
n = 0.0

def TimepixTagger():
    global triggerTime
    global scalarRepeat
    global eventA
    global prevTriggerTime
        
    # ADC value which increases on a pulser
    ADC104    = aq.adcArray[np.where(aq.adcArray['adc']==104)[0]]['val'][0]
    # ADC values which define the clock of the trigger
    shortTime = aq.adcArray[np.where(aq.adcArray['adc']==102)[0]]['val'][0]
    longTime  = aq.adcArray[np.where(aq.adcArray['adc']==103)[0]]['val'][0]
    # Calibration of the clock to roughly ns
    # Add this time to the list of times
    triggerTime += [8*(shortTime+65536*(longTime+65536*scalerRepeat))]
    
#
   #
      #
   #         
# 
    
    # if event contains epics information
    if(aq.epicsEvent==1):
        # check if the epics information has been updated since the previous epics event
        if(eventA!=aq.getEpicsPV('PPOL:TIMEPIXA:EVENT')):
            # Get the raw information out of the epics event
            eventA = aq.getEpicsPV('PPOL:TIMEPIXA:EVENT')
            nHitsA = aq.getEpicsPV('PPOL:TIMEPIXA:NHITS')
            encodedA = aq.getEpicsPV('PPOL:TIMEPIXA:ENCODED')
            
            # decode the data and create a data frame for timepix A
            TimepixAData = pd.DataFrame(Timepix.Decode(nHitsA,encodedA))     
            
            # calibrate the times and put them in the data frame
            TimepixAData['ctime']=16*TimepixAData['ToA'].astype(long)-(TimepixAData['FToA'].astype(int))
            
            # sort by the calibrated times
            TimepixAData=TimepixAData.sort_values(by='ctime')
            TimepixAData = TimepixAData.reset_index(drop=True)
            
            # determine the cluster groupings
            TimepixAData['tdiff']=TimepixAData['ctime']-TimepixAData['ctime'].shift() # get the time diffs between hits
            TimepixAData=TimepixAData.fillna(1000).astype(int)    # replace NaNs with 0s
            
            # call function to add new columns with total time of a cluster, number of hits, and group number
            newValues = TimepixAData.apply(find_clusters,axis=1)
            newValues.columns = ['group #','nhits','twalk']
            TimepixAData = pd.concat([TimepixAData, newValues], axis=1)
            
            # find the mean position of the cluster
            newValues = TimepixAData.apply(get_mean_pos,axis=1)
            newValues.columns = ['mean x','mean y']
            TimepixAData = pd.concat([TimepixAData,newValues], axis=1)
            
#
   #
      #
   #         
#              
            
            # If the trigger clock has looped add the loop length to the variables
            triggerTime2 = np.array(triggerTime,dtype=np.float)
            triggerWrap  = np.where(np.diff(triggerTime2)<0)[0]
            if(len(triggerWrap)>0):
                triggerTime2[triggerWrap[0]:] += 65536*65536*8
                
            # Determine which pulser trigger relates to the timepix event
            # This seems to get mixed up between the previous one and the one before that
            # Some are still wrong but enough right not to worry for now.
            timeSub = prevTriggerTime[-1]
            if((triggerTime2-timeSub)[-1]<1000000000 and len(prevTriggerTime)>1): 
                timeSub = prevTriggerTime[-2]
                
            # Subtract that previous time
            triggerTime2 -= timeSub
            # Convert the clock times into 1.5625ns.
            # The differnce between the timepix and trigger clocks seems to be by a factor 1.015595 or there abouts
            triggerTime2 /= 1.015595*1.5625
            
            # Coincidence index list
            coinIdx = []
            
            # Loop through the timepix times finding which ones are in coincidence with trigger events
            #for i, hit in enumerate(timesA):
                # Difference between the trigger times and the Timepix times
            #    tdiff = triggerTime2-hit
                # If there is a coincidence add the event index to the list
                # For some reason there are two peaks...
                # Could have a separate list of random events which could be scaled and subtracted like with the tagger.
            #    if (len(tdiff[np.abs(tdiff)<10000])>0 or len(tdiff[np.abs(tdiff-263000)<10000])>0):
            #        coinIdx +=  [i]

#
   #
      #
   #         
#                       
            
            # making a data frame where the entries are clusters rather than hits
            ClustersA = pd.DataFrame()
            
            nh = []
            tw = []
            mx = []
            my = []
            
            g = TimepixAData['group #'][0]
            for i in range(len(TimepixAData)):
                if (TimepixAData['group #'][i]!=g):
                    nh += [TimepixAData['nhits'][i-1]+1]
                    tw += [TimepixAData['twalk'][i-1]]
                    mx += [TimepixAData['mean x'][i-1]]
                    my += [TimepixAData['mean y'][i-1]]
                    g+=1
            if (g==TimepixAData['group #'][i]):
                nh += [TimepixAData['nhits'][i]+1]
                tw += [TimepixAData['twalk'][i]]
                mx += [TimepixAData['mean x'][i]]
                my += [TimepixAData['mean y'][i]]
            
            
            
            #print(TimepixAData['mean x'][i])
            
            ClustersA['cluster #'] = list(range(1,len(mx)+1))
            ClustersA['nhits'] = nh
            ClustersA['tspan'] = tw
            ClustersA['mean x'] = mx
            ClustersA['mean y'] = my
            
            #hist_posA.fill_array(ClustersA[['mean x','mean y']])

#
   #
      #
   #         
#                       

            print(ClustersA)
            #print(TimepixAData)
            
            eventA = aq.getEpicsPV('PPOL:TIMEPIXA:EVENT')
            
    if(aq.eventNo%10000==0):                          # print a processing statement every 5000 events
        print("Events Processed: "+str(aq.eventNo))
    
aq.runFunction(TimepixTagger,0,500000)

     cluster #  nhits  tspan  mean x  mean y
0            1      1      0   14.00   25.00
1            2      2     10  171.00  114.50
2            3      3     37  152.67  105.33
3            4      2     19   92.50  113.00
4            5      1      0  148.00  230.00
5            6      1      0  181.00  167.00
6            7      2     21   98.50   98.00
7            8      3     19  230.33    0.33
8            9      1      0  192.00  184.00
9           10      2     13  251.50  178.00
10          11      1      0  167.00   72.00
11          12      7     46  215.14  126.29
12          13      1      0  185.00  175.00
13          14      3     30  152.33   90.33
14          15      2     19  118.00  157.50
15          16      2     35  119.50  130.00
16          17      1      0  143.00  170.00
17          18      2     13  156.00  121.50
18          19      2     18  157.00  215.50
19          20      1      0  150.00  249.00
20          21      3     24  142.33  118.33
21        

     cluster #  nhits  tspan  mean x  mean y
0            1      3     29  177.33  225.00
1            2      2     12  194.00   55.50
2            3      1      0  217.00  155.00
3            4      6     20  212.83  229.83
4            5      1      0  192.00   11.00
5            6      3     25  203.33  238.33
6            7      1      0  189.00  166.00
7            8      2     33  154.50  156.00
8            9      2     22  141.00  225.50
9           10      6     26  178.33  191.83
10          11      2     23  228.00  106.50
11          12      2      8  136.00  160.50
12          13      2     20  174.00  204.50
13          14      2     31  174.50  110.50
14          15      1      0  172.00  215.00
15          16      6     27  185.83  248.67
16          17      6     36  123.17   88.33
17          18      1      0  207.00  205.00
18          19      1      0  188.00   77.00
19          20      2     20  186.00  251.50
20          21      3     31   81.33  141.00
21        

     cluster #  nhits  tspan  mean x  mean y
0            1      6     11  234.17   28.17
1            2      1      0   22.00  219.00
2            3      2     12  182.00  181.50
3            4      6     27  214.00  249.83
4            5      1      0  182.00  141.00
5            6      1      0  174.00  156.00
6            7      2     11  156.00   14.50
7            8      2     29  138.00  194.50
8            9      2     32  198.50  228.00
9           10      2     34  159.00  234.50
10          11      2     42  150.00   33.50
11          12      1      0  172.00  200.00
12          13      2     24  212.00  233.50
13          14      1      0  234.00  168.00
14          15      2      2  192.00  179.50
15          16      2      4  118.50  163.00
16          17      1      0  168.00  168.00
17          18      2     26  157.50  188.00
18          19      1      0  165.00  251.00
19          20      2     55  148.50  121.50
20          21      2     61  220.50  163.00
21        

     cluster #  nhits  tspan  mean x  mean y
0            1      3     41    7.67  230.67
1            2      1      0  178.00  247.00
2            3      1      0  219.00   98.00
3            4      2     18   98.00   30.50
4            5      1      0   83.00    0.00
5            6      1      0  153.00  158.00
6            7      3     54  174.67  167.33
7            8      1      0  148.00  119.00
8            9      1      0  226.00  206.00
9           10      1      0  226.00  126.00
10          11      1      0  122.00  188.00
11          12      1      0  166.00  238.00
12          13      1      0  171.00   67.00
13          14      1      0  124.00  141.00
14          15      2     52  148.50  253.00
15          16      1      0  229.00  235.00
16          17      2      2  102.00  231.50
17          18      6     35  247.83  203.00
18          19      4     27  212.25  162.25
19          20      2     30  145.00  179.50
20          21      1      0  116.00   59.00
21        

     cluster #  nhits  tspan  mean x  mean y
0            1      1      0  184.00   30.00
1            2      3     37  203.33    3.33
2            3      2      4  214.50   72.00
3            4      2     10  136.00  172.50
4            5      1      0  248.00  140.00
5            6      2     56  170.00  132.50
6            7      2     28  146.00  215.50
7            8      1      0  169.00  187.00
8            9      1      0  114.00  102.00
9           10      2     18  231.00  249.50
10          11      4     43  195.50  156.50
11          12      3     48  160.67  216.67
12          13      1      0  143.00   28.00
13          14      2     14  177.00   47.50
14          15      1      0  172.00  168.00
15          16      2     13  199.00  228.50
16          17      1      0  203.00   78.00
17          18      2      6  167.50  109.00
18          19      2     14  174.00  144.50
19          20      3     25  210.33   26.33
20          21      1      0  208.00    1.00
21        

...

     cluster #  nhits  tspan  mean x  mean y
0            1      2     47  175.50  223.00
1            2      3     47    7.67  159.67
2            3      1      0  197.00    8.00
3            4      2     28  177.00   76.50
4            5      1      0  168.00   82.00
5            6      1      0  178.00  228.00
6            7      2     27  185.00  193.50
7            8      8     39  187.88   95.25
8            9      4     50  200.50  128.50
9           10      2      0  195.00   74.50
10          11      1      0  178.00  124.00
11          12      3     45  249.33  232.33
12          13      2     29  181.50  159.00
13          14      2      1  217.50   92.00
14          15      1      0    3.00   86.00
15          16      5     25  201.80   71.60
16          17      2     27  237.50  171.00
17          18      1      0  187.00   92.00
18          19      1      0  162.00  247.00
19          20      1      0  154.00   96.00
20          21      2     28  159.00  234.50
21        

     cluster #  nhits  tspan  mean x  mean y
0            1      1      0  210.00  226.00
1            2      2     25  175.00  134.50
2            3      2     27  212.00  188.50
3            4      2     32  222.50  154.00
4            5      2      3  148.00  245.50
5            6      2     14  216.00  227.50
6            7      1      0  129.00  163.00
7            8      1      0  162.00  151.00
8            9      2      8  182.50  103.00
9           10      1      0  170.00  189.00
10          11      2      2  194.50   58.00
11          12      1      0  167.00  175.00
12          13      2     51  132.50  105.00
13          14      2     16  178.00  145.50
14          15      3     45  165.33   44.33
15          16      1      0  104.00  252.00
16          17      2     45  123.50  207.00
17          18      3     13  105.00  243.00
18          19      1      0  244.00  150.00
19          20      1      0  185.00  229.00
20          21      1      0  232.00  189.00
21        

     cluster #  nhits  tspan  mean x  mean y
0            1      3     20  217.67  243.67
1            2      3     23  143.67  141.67
2            3      1      0  199.00  211.00
3            4      1      0  240.00  125.00
4            5      1      0  220.00    7.00
5            6      1      0  164.00  188.00
6            7      4     43   84.00   37.75
7            8      2     30  244.00   42.50
8            9      1      0  187.00   75.00
9           10      1      0  108.00  229.00
10          11     10     44  142.80  203.80
11          12      1      0  206.00  225.00
12          13      1      0  148.00  227.00
13          14      3     36  226.33  179.00
14          15      1      0  169.00   87.00
15          16      2     43  160.00   44.50
16          17      3     20  200.00  138.67
17          18      2     42  159.00  193.50
18          19      1      0   80.00  216.00
19          20      1      0  209.00  213.00
20          21      2     47  171.50  198.00
21        

Events Processed: 100000
     cluster #  nhits  tspan  mean x  mean y
0            1      2     41  182.50  246.00
1            2      3     11  198.67   17.00
2            3      2     19  225.00  167.50
3            4      2     45  239.00  136.50
4            5      2     38  146.50  132.00
5            6      1      0  167.00   18.00
6            7      1      0  170.00   12.00
7            8      2     21  168.50  146.00
8            9      1      0  133.00  253.00
9           10      3     33  245.33   75.33
10          11      2     13  135.00   21.50
11          12      1      0  198.00   68.00
12          13      1      0  179.00  231.00
13          14      3     11  156.33   84.67
14          15      1      0   12.00  233.00
15          16      3     37  174.33  242.33
16          17      1      0  240.00  114.00
17          18      1      0  220.00  130.00
18          19      2     26  195.00  173.50
19          20      1      0   92.00   75.00
20          21      2     34  

     cluster #  nhits  tspan  mean x  mean y
0            1      2      2  127.00    3.50
1            2      2     27  218.00   86.50
2            3      1      0  167.00  147.00
3            4      3     16   64.00   70.00
4            5      1      0  160.00  217.00
5            6      1      0  144.00  168.00
6            7      1      0  216.00  255.00
7            8      9     49  225.89  128.78
8            9      1      0  179.00   81.00
9           10      1      0  170.00  194.00
10          11      1      0  236.00   71.00
11          12      1      0  120.00  139.00
12          13      5     34  206.20  138.40
13          14      1      0  168.00  235.00
14          15      2     14  219.50   36.00
15          16      1      0  100.00   29.00
16          17      2     10  199.00  253.50
17          18      2     31  179.50  229.00
18          19      3     29  177.67   66.67
19          20      2     41  224.50  113.00
20          21      1      0  154.00  216.00
21        

In [3]:
# this function gets the mean position of a cluster
def get_mean_pos(data):
    global sumx        # used to sum up the x positions
    global sumy        # used to sum up the y positions
    global meanx       # working mean x position
    global meany       # working mean y position
    global gn          # cluster group number
    global n           # number of hits in the cluster so far

    if (data['group #']!=gn):       # if we're on the next cluster, reset everything
        sumx = 0.0
        sumy = 0.0
        n = 0.0
        gn += 1
    
    sumx += data['x']
    sumy += data['y']
    n+=1.0
    meanx = round(sumx/n,2)
    meany = round(sumy/n,2)
    
    return pd.Series([meanx,meany])

In [4]:
# this gets applied to every row in the dataframe input, and determines which cluster that hit belongs to
def find_clusters(data):
    global counter    # time from the first hit in the counter
    global hits       # number of hits in the cluster
    global group      # allocate seperate group numbers for each cluster
    
    if (data['tdiff'] > 100):   # if too far, start new cluster cluster
        counter = 0            # zero the time counter
        hits = 0               # zero the hits for the cluster
        group += 1             # increment the cluster counter
    else:                            # if we can continue the cluster
        counter += data['tdiff']     # increment the counter by the time difference
        hits += 1                    # increment the hits in this cluster by one
    
    return pd.Series([group,hits,counter])

Plot those histograms:

In [169]:

ROOT.enableJSVis()
c1 = ROOT.TCanvas("my1","The Canvas Title",1000,1000)
hist_posA.Draw("colz")
hist_posA.GetXaxis().SetTitle("mean x of cluster")
hist_posA.GetYaxis().SetTitle("mean y of cluster")
#c1.Draw()

Another way? Testing:

def TimepixTagger():
    global eventA
    
    # if event contains epics information
    if(aq.epicsEvent==1):
        # check if the epics information has been updated since the previous epics event
        if(eventA!=aq.getEpicsPV('PPOL:TIMEPIXA:EVENT')):
            # Get the raw information out of the epics event
            eventA = aq.getEpicsPV('PPOL:TIMEPIXA:EVENT')
            nHitsA = aq.getEpicsPV('PPOL:TIMEPIXA:NHITS')
            encodedA = aq.getEpicsPV('PPOL:TIMEPIXA:ENCODED')
            
            # decode the data and create a data frame for timepix A
            TimepixAData = pd.DataFrame(Timepix.Decode(nHitsA,encodedA))     
            
            # calibrate the times and put them in the data frame
            TimepixAData['ctime']=16*TimepixAData['ToA'].astype(long)-(TimepixAData['FToA'].astype(int))
            
            # sort by the calibrated times
            TimepixAData=TimepixAData.sort_values(by='ctime')
            
            # determine the cluster groupings
            TimepixAData['tdiff']=TimepixAData['ctime']-TimepixAData['ctime'].shift() # get the time diffs between hits
            TimepixAData=TimepixAData.fillna(1000).astype(int)    # replace NaNs with 0s
            
            # call function to add new columns with total time of a cluster, number of hits, and group number
            newValues = TimepixAData.apply(find_clusters,axis=1)
            newValues.columns = ['twalk','nhits','group #']
            TimepixAData = pd.concat([TimepixAData, newValues], axis=1)
        
            
            # like Simon's:
            
            ClustersA=pd.DataFrame()
            # indices where diff>100
            ClustersA['break'] = np.concatenate([np.where(TimepixAData['tdiff']>100)[0]])
            
            lengA = []
            diffA = []
            timesA  = []
            x   = []
            y   = []
 
            if(len(TimepixAData['ctime'])==0): 
                ClustersA['break'] = []
            
            for ssi in zip(ClustersA['break'][:-1],ClustersA['break'][1:]):
                lengA  += [ssi[1]-ssi[0]]  # cluster size
                diffA += [TimepixAData['ctime'][ssi[1]]-TimepixAData['ctime'][ssi[0]]]  # time differences
                timesA += [TimepixAData['ctime'][ssi[0]]] # Time of first hit
                x += [TimepixAData['x'][ssi[0]]]    # position of first hit in cluster
                y += [TimepixAData['y'][ssi[0]]]    #(Mean position would probably improve)
                
            ClustersA['leng']=np.concatenate([[0],lengA])    
            ClustersA['tdiff']=np.concatenate([[0],diffA])
            ClustersA['tfirst']=np.concatenate([[0],timesA])
            ClustersA['x1']=np.concatenate([[0],x])
            ClustersA['y1']=np.concatenate([[0],y])
            
            
            
    
            print(ClustersA)
            #print(TimepixAData)
    
    if(aq.eventNo%10000==0):                          # print a processing statement every 5000 events
        print("Events Processed: ",aq.eventNo)
    
aq.runFunction(TimepixTagger,0,5000000)